In [41]:
# Importing needed libraries
from tkinter import *
from PIL import ImageTk,Image
import re
import cv2
import time

# Defining the paths
path = "/home/mooeyad/Downloads/project/gui/graphics"
path_saving = "project/saved_data"

# Defining and setting parameters for the root GUI window
root = Tk()
root.title("StepScope")   #root window title
root.geometry("700x300")  #Startup Resolution
#root.iconbitmap(path+"/icon_c.ico") #Program icon


# Load all images that will be used
HeatMap_cv = cv2.imread(path+"/heatmap.png", cv2.IMREAD_UNCHANGED)

# Convert the image to a PIL Image object
HeatMap_pil = Image.fromarray(cv2.cvtColor(HeatMap_cv, cv2.COLOR_BGRA2RGBA))


# Create a Tkinter PhotoImage object from the PIL Image object
HeatMap_tk = ImageTk.PhotoImage(HeatMap_pil, format="RGBA")

# Importing the Gait Cycle Images
gaitNA_tk = ImageTk.PhotoImage(Image.open(path+"/GaitPhases/NA.png"), format="RGBA")
gaitIC_tk = ImageTk.PhotoImage(Image.open(path+"/GaitPhases/IC.png"), format="RGBA")
gaitFF_tk = ImageTk.PhotoImage(Image.open(path+"/GaitPhases/FF.png"), format="RGBA")
gaitMS_tk = ImageTk.PhotoImage(Image.open(path+"/GaitPhases/MS.png"), format="RGBA")
gaitHL_tk = ImageTk.PhotoImage(Image.open(path+"/GaitPhases/HL.png"), format="RGBA")
gaitTO_tk = ImageTk.PhotoImage(Image.open(path+"/GaitPhases/TO.png"), format="RGBA")


# importing and visualizing the logo on the root window
logo_img = ImageTk.PhotoImage(Image.open(path+"/LOGO_colorR.png")) 
logo_label = Label(image = logo_img)
logo_label.grid(row=0,column=0)

###################  VARIABLES INITIALIZATION #################
P_Name = ""                                   # Patient Name
P_Gender = ""                                 # Patient Gender
P_Age = 0                                     # Patient Age
P_Height = 0                                  # Patient Height
P_Weight = 0                                  # Patient Weight

offsetX= 1                                    # X-axis offset for grid allignment
offsetY= 0                                    # Y-axis offset for grid allignment

DataConfirmed = False                         # Boolean for data confirmation
Transitioned = False                          # Boolean for window transitioning
StopWatchRunning = False                      # Boolean for to run the stop watch

IMU_Shin = 0                                  # IMU Shin Readings
IMU_Thigh = 0                                 # IMU Thigh Reading
Pressure_Readings = [0,0,0,0,0]               # FSR Readings

GenderChoices = ["Male","Female"]             # Gender Choices for drop down menu

GaitStates = ["N/A","Initial Contact","Foot Flat","Midstance","Heel lift" , "Toe off", "Acceleration" ,"Midswing" , "Deceleration" ]
                                              # Gait Phases List
    
gaitphases_tk = [gaitNA_tk , gaitIC_tk , gaitFF_tk , gaitMS_tk , gaitHL_tk , gaitTO_tk]
                                              # Gait Phases Images List 

NewWindow = None                              # Variable to store new window

stopwatch_label = None                        # Stopwatch label

index = 0                                     # Index to access the previous lists

######################################################################

# Funtion for a number only string
def contains_only_numbers(input_string):
    pattern = "^[0-9]+$"  # match string with only numbers
    return not(bool(re.match(pattern, input_string)))

# Function to clear a grid area
def clear_grid_area(row, column , window):
    for widget in window.grid_slaves(row, column):
        widget.destroy()



In [42]:
#Stop watch functionlity 
def start_stopwatch():
    global start_time
    global StopWatchRunning
    
    start_time = time.time() # get the current time when the stopwatch is started
    StopWatchRunning = True
    update_stopwatch() # start updating the stopwatch label

def update_stopwatch():
    global NewWindow
    global stopwatch_label
    global StopWatchRunning
    global update_id
    
    if StopWatchRunning:
        elapsed_time = time.time() - start_time # calculate the elapsed time
        #clear_grid_area(7,0,NewWindow)
        stopwatch_label = Label(NewWindow, text=format_time(elapsed_time), font=("Arial", 12))
        stopwatch_label.grid(row=7,column=0)
        update_id =stopwatch_label.after(100, update_stopwatch) # schedule the update function to be called after 100ms (0.1 second)
    else:
        pass

def format_time(elapsed_time):
    minutes = int(elapsed_time / 60)
    seconds = int(elapsed_time - minutes * 60)
    milliseconds = int((elapsed_time - minutes * 60 - seconds) * 100)
    return f"{minutes:02d}:{seconds:02d}:{milliseconds:02d}"

def stop_stopwatch():
    global StopWatchRunning

    stopwatch_label.after_cancel(update_id) # cancel the scheduled update function
    StopWatchRunning = False


In [43]:
#Simple Test Function for debugging
def test():
    UpdateGaitPhase(2)

In [44]:
# Function that will excute from the confirm button
def ConfirmClick():
    global P_Name , P_Age , P_Height , P_Weight , P_Gender
    global DataConfirmed
    global root

    # Get the entered data in the text boxes
    P_Name= (NameEntry.get())
    P_Gender = GenderVar.get()
    P_Age = (AgeEntry.get())
    P_Height = (HeightEntry.get())
    P_Weight = (WeightEntry.get())
    
    # Check for the data validity 
    if (P_Name == "") or (P_Age == "") or (P_Height == "") or (P_Weight == "")\
    or contains_only_numbers(P_Age) or contains_only_numbers(P_Height) or \
    contains_only_numbers(P_Weight) :
        # Data entered is incomplete / invalid
        clear_grid_area(6+offsetX,0+offsetY,root)
        clear_grid_area(7+offsetX,0+offsetY,root)
        ErrorLabel = Label(root,text="Please Enter the data correctly")
        ErrorLabel.grid(row=6+offsetX,column=0+offsetY)
        DataConfirmed = False

    else:
        # Data entered is valid
        clear_grid_area(6+offsetX,0+offsetY,root)
        ConfirmationLabel = Label(root,text="Data is Entered")
        ConfirmationLabel.grid(row=6+offsetX,column=0+offsetY)
        P_StrData = "Patient name is "+P_Name+" age is " +str(P_Age)+ ", is " +\
        str(P_Height)+" cm tall and weighs "+str(P_Weight)+" KG " +"Gender is "+ P_Gender
        PatientDataLabel = Label(root,text=P_StrData)
        PatientDataLabel.grid(row=7+offsetX,column=0+offsetY)
        DataConfirmed = True 
        
# Function that will excute from the next button        
def NextClick():

    global DataConfirmed , Transitioned
    global NewWindow
    global root

    # Check for Confirmed data
    if DataConfirmed:
        # Data is Confirmed
        clear_grid_area(6+offsetX,0+offsetY,root)
        clear_grid_area(7+offsetX,0+offsetY,root)
        TransitionLabel = Label(root,text="Transitioning to the next stage....")
        TransitionLabel.grid(row=6+offsetX,column=0+offsetY)
        # Start a new window for data display
        DataWindow = Toplevel(root)
        DataWindow.title = ("Data Display") 
        DataWindow.geometry("800x600")
        NewWindow = DataWindow
        DrawDataWindow(DataWindow)
        
    else:
        # Data is not Confirmed yet
        clear_grid_area(6+offsetX,0+offsetY)
        clear_grid_area(7+offsetX,0+offsetY)
        ErrorLabel = Label(root,text="Please Enter the data and confirm it first")
        ErrorLabel.grid(row=6+offsetX,column=0+offsetY)
        
# Creating entry text boxes 
NameEntry = Entry(root)
AgeEntry = Entry(root)
HeightEntry = Entry(root)
WeightEntry = Entry(root)

# Creating a drop menu for the gender choice
GenderVar=StringVar(root)
GenderVar.set("Male")
GenderChoice = OptionMenu(root , GenderVar ,*GenderChoices)

# Creating a Label for the required text
NameLabel   =  Label(root,text="Patient Name:")
AgeLabel    =  Label(root,text="Patient Age:")
HeightLabel =  Label(root,text="Patient Hieght:")
WeightLabel =  Label(root,text="Patient Weight:")
GenderLabel =  Label(root,text="Patient Gender:")

# Creating the Confrim data button
ConfirmButton = Button(root, text="Confirm Data" , padx = 50 , command = ConfirmClick)

# Creating the next button
NextButton    = Button(root, text="Next" , padx = 50 , command = NextClick)

# Creating Update button (Debugging)
updatebutton  = Button(root, text="update" , padx = 50 , command = test)


In [45]:
# Assigning every label and button a place on the window grid
####################Labels###################
NameLabel.grid(row=0+offsetX,column=0+offsetY)
AgeLabel.grid(row=1+offsetX,column=0+offsetY)
HeightLabel.grid(row=2+offsetX,column=0+offsetY)
WeightLabel.grid(row=3+offsetX,column=0+offsetY)
GenderLabel.grid(row=4+offsetX,column=0+offsetY)

####################Entries###################
NameEntry.grid(row=0+offsetX,column=1+offsetY)
AgeEntry.grid(row=1+offsetX,column=1+offsetY)
HeightEntry.grid(row=2+offsetX,column=1+offsetY)
WeightEntry.grid(row=3+offsetX,column=1+offsetY)
GenderChoice.grid(row=4+offsetX,column=1+offsetY)

####################Buttons###################
ConfirmButton.grid(row=5+offsetX,column=0+offsetY)
NextButton.grid(row=5+offsetX,column=1+offsetY)
updatebutton.grid(row=7,column=1)


In [46]:
# Function to record the sent values and patient data in an np array and save it to the disk as csv file when stop record is clicked
# It must start a timer when clicked to show the recording time
def RecordClick():
    
    global NewWindow
    global stopwatch_label
    
    stopwatch_label = Label(NewWindow, text="00:00:00", font=("Arial", 12))
    stopwatch_label.grid(row=7,column=0)
    start_stopwatch()
    clear_grid_area(6,1,NewWindow)
    # Creating Start Record button 
    stopbutton  = Button(NewWindow, text="stop record" , padx = 50 , command = StopClick)
    stopbutton.grid(row=6,column=1)


def StopClick():
    
    global NewWindow
    
    stop_stopwatch()
    clear_grid_area(6,1,NewWindow)
    # Creating Stop Record button
    SavingLabel = Label(NewWindow,text="File saved to path: "+ path_saving)
    SavingLabel.grid(row=6,column=1)


In [47]:
# Function that draws the content of the new display data windows for the first time
def DrawDataWindow(DataWindow):
    
    # Connection Status Labels and Grid assignment
    ConnectionLabel1 = Label(DataWindow ,text="Connection 1:")
    ConnectionLabel2 = Label(DataWindow ,text="Connection 2:")
    ConnectionLabel1.grid(row=0,column=0)
    ConnectionLabel2.grid(row=1,column=0)
    StatusLabel1 = Label(DataWindow ,text="Failed" ,fg ='#ff0000')
    StatusLabel2 = Label(DataWindow ,text="Failed" ,fg='#ff0000')
    StatusLabel1.grid(row=0,column=1)
    StatusLabel2.grid(row=1,column=1)
    
    # IMU sensors readings Labels and Grid assignment
    IMU1Label = Label(DataWindow ,text="IMU Thigh:")
    IMU2Label = Label(DataWindow ,text="IMU Shin:")
    IMU1Label.grid(row=2,column=0)
    IMU2Label.grid(row=3,column=0)
    IMU1ValueLabel = Label(DataWindow ,text=str(IMU_Thigh))
    IMU2ValueLabel = Label(DataWindow ,text=str(IMU_Shin))
    IMU1ValueLabel.grid(row=2,column=1)
    IMU2ValueLabel.grid(row=3,column=1)
    
    # Knee Angle calculation and Grid assingment
    KneeAngleLabel = Label(DataWindow ,text="Knee angle:")
    KneeAngleLabel.grid(row=4,column=0)
    KneeAngleValueLabel = Label(DataWindow ,text=str(abs(IMU_Thigh-IMU_Shin)))
    KneeAngleValueLabel.grid(row=4,column=1)
    
    # Gait Phase prediction Labels and Grid assignment
    GaitPhaseLabel = Label(DataWindow ,text="Predicted Gait Phase:")
    GaitPhaseLabel.grid(row=5,column=0)
    GaitPhaseValueLabel = Label(DataWindow ,text=GaitStates[index],fg='#ff0000')
    GaitPhaseValueLabel.grid(row=5,column=1)
    Gaitimg_Label = Label(DataWindow , image=gaitphases_tk[index])
    Gaitimg_Label.grid(row=9,column=0)
    
    # Heatmap Label and Grid assignment
    HeatMapImg_Label = Label(DataWindow , image=HeatMap_tk)
    HeatMapImg_Label.grid(row=9,column=1)
    
    # Record button and Grid assignment
    Recordbutton = Button(DataWindow, text="Record" , padx = 50 , command= RecordClick)
    Recordbutton.grid(row=6,column=0)


In [48]:
#Update functions to update each value in the new display data window
def UpdateStatus1(state):        #Update Connection 1
    global NewWindow
    if state:
        StatusLabel1 = Label(NewWindow ,text="Successful" ,fg ='#00ff00')
        StatusLabel1.grid(row=0,column=1)
    else:
        StatusLabel1 = Label(NewWindow ,text="Failed" ,fg ='#ff0000')
        StatusLabel1.grid(row=0,column=1)
    
def UpdateStatus2(state):        #Update Connection 2
    global NewWindow
    if state:
        StatusLabel1 = Label(NewWindow ,text="Successful" ,fg ='#00ff00')
        StatusLabel1.grid(row=1,column=1)
    else:
        StatusLabel1 = Label(NewWindow ,text="Failed" ,fg ='#ff0000')
        StatusLabel1.grid(row=1,column=1)

def UpdateIMU1Value(Value):        #Update IMU Thigh readings
    global NewWindow
    IMU1ValueLabel = Label(NewWindow ,text=str(Value))
    IMU_Thigh = Value
    IMU1ValueLabel.grid(row=2,column=1)
    
def UpdateIMU1Value(Value):        #Update IMU Shin readings
    global NewWindow
    IMU2ValueLabel = Label(NewWindow ,text=str(Value))
    IMU_Shin = Value
    IMU2ValueLabel.grid(row=3,column=1)

def UpdateKneeAnglee():            #Update Knee Angle calculation
    global NewWindow
    KneeAngleValueLabel = Label(DataWindow ,text=str(abs(IMU_Thigh-IMU_Shin)))
    KneeAngleValueLabel.grid(row=4,column=1)

def UpdateGaitPhase(index):        #Update the predicted gait phase
    global NewWindow
    GaitPhaseLabel = Label(NewWindow ,text="Predicted Gait Phase:")
    GaitPhaseLabel.grid(row=5,column=0)
    GaitPhaseValueLabel = Label(NewWindow ,text=GaitStates[index],fg='#ff00ff')
    GaitPhaseValueLabel.grid(row=5,column=1)
    Gaitimg_Label = Label(NewWindow , image=gaitphases_tk[index])
    Gaitimg_Label.grid(row=9,column=0)

def UpdateHeatMap(FSR_Arr):         #Update the heat map
    global NewWindow
    pass

In [49]:
#looping the gui
root.mainloop()